In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# import seaborn as sns
pd.options.plotting.backend = "plotly"
# pd.options.plotting.backend = "matplotlib"
import datetime
# import math
# import pandas_profiling as pp

In [2]:
# dataset path
alerts_path = r'D:\DSLS\Mini project\DS\clean_code\data\raw\aggregate_alerts_Kota Bogor.csv'
median_irregularities_path = r'D:\DSLS\Mini project\DS\clean_code\data\raw\aggregate_median_irregularities_Kota Bogor.csv'
median_jams_path = r'D:\DSLS\Mini project\DS\clean_code\data\raw\aggregate_median_jams_Kota Bogor.csv'

# Data understanding

## Jams

In [11]:
df_jams = pd.read_csv(median_jams_path)
df_jams = df_jams.drop(['Unnamed: 0'], axis=1)
df_jams.head(3)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
0,2022-07-06 00:00:00.000,32.71,KOTA BOGOR,Pahlawan,2,784.0,62.0,20.310001,4,33469021,2022-07-06,2.0,"MULTILINESTRING ((106.797269 -6.608899, 106.79..."
1,2022-07-06 00:00:00.000,32.71,KOTA BOGOR,Tol Lingkar Luar Bogor,2,1314.0,63.0,37.710000,2,33469023,2022-07-06,2.0,"LINESTRING (106.787027 -6.559782, 106.787701 -..."
2,2022-07-06 00:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),5,167.0,-1.0,0.000000,61,33469019,2022-07-06,5.0,"MULTILINESTRING ((106.800516 -6.561997, 106.80..."


In [ ]:
# analisis null value
df_jams.isna().sum().plot(kind='bar')

In [ ]:
# analisis statistik
df_jams.describe()

,Unnamed: 0,kemendagri_kabupaten_kode,level,median_length,median_delay,median_speed_kmh,total_records,id,median_level
count,102322.000000,1.023220e+05,102322.000000,102322.000000,102322.000000,102322.000000,102322.000000,1.023220e+05,102322.000000
mean,33353.589345,3.271000e+01,2.433758,840.802794,131.969029,14.084945,17.575673,3.539966e+07,2.433758
std,23566.968434,2.131639e-14,1.002043,549.866997,109.430777,7.975882,20.682176,1.092828e+06,1.002043
min,0.000000,3.271000e+01,1.000000,13.000000,-1.000000,0.000000,1.000000,3.346902e+07,1.000000
25%,12790.000000,3.271000e+01,2.000000,444.000000,75.000000,7.885000,4.000000,3.449331e+07,2.000000
50%,28202.500000,3.271000e+01,2.000000,727.000000,98.000000,13.060000,10.000000,3.539202e+07,2.000000
75%,53782.750000,3.271000e+01,3.000000,1138.000000,145.500000,19.090000,23.000000,3.637102e+07,3.000000
max,79363.000000,3.271000e+01,5.000000,8307.000000,2754.000000,63.614998,600.000000,3.727920e+07,5.000000


In [13]:
# analisis kolom
print(df_jams.info())
df_jams['median_level'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102322 entries, 0 to 102321
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   time                       102322 non-null  object 
 1   kemendagri_kabupaten_kode  102322 non-null  float64
 2   kemendagri_kabupaten_nama  102322 non-null  object 
 3   street                     100529 non-null  object 
 4   level                      102322 non-null  int64  
 5   median_length              102322 non-null  float64
 6   median_delay               102322 non-null  float64
 7   median_speed_kmh           102322 non-null  float64
 8   total_records              102322 non-null  int64  
 9   id                         102322 non-null  int64  
 10  date                       102322 non-null  object 
 11  median_level               102322 non-null  float64
 12  geometry                   102322 non-null  object 
dtypes: float64(5), int64(3), obje

In [14]:
# sampling jalan dengan data terbanyak (3 sample jalan)
df_jams_clean = df_jams[df_jams['street'].notna()]
street_list = pd.DataFrame(columns=['street_name', 'total'])

for street in (df_jams_clean['street'].unique()):
  street_list = pd.concat([street_list, pd.DataFrame.from_records([{'street_name':street, 'total': len(df_jams_clean.loc[(df_jams['street'] == street)])}])])

street_list.sort_values(by='total', ascending=False).head(5)

,street_name,total
0,N9 Raden KH Abdullah bin Nuh,3658
0,N9 Jalan Raya Pajajaran,3625
0,N9 Jalan Raya Dramaga,3174
0,N9 Jalan Raya Tajur,3054
0,N8 Jalan Raya Bogor,2954


In [19]:
for i, street in enumerate(['N9 Raden KH Abdullah bin Nuh', 'N9 Jalan Raya Pajajaran', 'N9 Jalan Raya Dramaga']):
  print(i, street)
  df_jams_sample = df_jams.loc[(df_jams['street'] == street)]
  df_jams_sample = df_jams_sample.drop(['kemendagri_kabupaten_kode', 'kemendagri_kabupaten_nama',
  'street', 'level', 'total_records', 'id', 'date', 'geometry'], axis=1)
  df_jams_sample['time'] = pd.to_datetime(df_jams_sample['time'])
  df_jams_agg = df_jams_sample.groupby(pd.Grouper(key="time", freq='1H')).agg({
    'median_length':np.median,
    'median_delay':np.mean,
    'median_speed_kmh':np.median,
    'median_level':np.mean})
  df_jams_agg['median_level'] = pd.to_numeric(df_jams_agg['median_level'].round(0))
  df_jams_agg.to_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_{}.csv'.format(i))

0 N9 Raden KH Abdullah bin Nuh
1 N9 Jalan Raya Pajajaran
2 N9 Jalan Raya Dramaga


In [17]:
# visualisasi histogram
for i, street in enumerate(['N9 Raden KH Abdullah bin Nuh', 'N9 Jalan Raya Pajajaran', 'N9 Jalan Raya Dramaga']):
  print(i, street)
  df_jams_sample = df_jams.loc[(df_jams['street'] == street)]
  df_jams_sample = df_jams_sample.drop(['kemendagri_kabupaten_kode', 'kemendagri_kabupaten_nama',
  'street', 'level', 'total_records', 'id', 'date', 'geometry'], axis=1)
  df_jams_sample['time'] = pd.to_datetime(df_jams_sample['time'])
  for col in df_jams_sample.columns:
    print(street, col)
    df_jams_sample[col].hist()
    plt.axvline(df_jams_sample[col].mean(), color='red', linestyle='dashed', linewidth=1)
    plt.axvline(df_jams_sample[col].median(), color='black', linestyle='dashed', linewidth=1)
    plt.show()

## Alerts

In [3]:
df_alerts = pd.read_csv(alerts_path)
df_alerts.head(3)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
0,5930177,2022-07-06 00:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,"[106.80051600000002, -6.561997000000004]",61,2022-07-06
1,5930178,2022-07-06 00:00:00.000,32.71,KOTA BOGOR,NaN,WEATHERHAZARD,"[106.80159300000008, -6.555780000000009]",61,2022-07-06
2,5930179,2022-07-06 01:00:00.000,32.71,KOTA BOGOR,KH Soleh Iskandar (Jalur Lambat),ROAD_CLOSED,"[106.80051600000002, -6.561997000000004]",60,2022-07-06


In [22]:
# analisis null value
df_alerts.isna().sum().plot(kind='bar')

In [23]:
# analisis statistik
df_alerts.describe()

,id,kemendagri_kabupaten_kode,total_records
count,9.762000e+03,9.762000e+03,9762.000000
mean,6.077419e+06,3.271000e+01,40.557160
std,8.243814e+04,1.421158e-14,33.080946
min,5.930177e+06,3.271000e+01,1.000000
25%,6.012826e+06,3.271000e+01,17.000000
50%,6.079400e+06,3.271000e+01,31.000000
75%,6.147232e+06,3.271000e+01,60.000000
max,6.220241e+06,3.271000e+01,727.000000


In [25]:
# analisis kolom
print(df_alerts.info())
df_alerts['type'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9762 entries, 0 to 9761
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         9762 non-null   int64  
 1   time                       9762 non-null   object 
 2   kemendagri_kabupaten_kode  9762 non-null   float64
 3   kemendagri_kabupaten_nama  9762 non-null   object 
 4   street                     8827 non-null   object 
 5   type                       9762 non-null   object 
 6   avg_location               9762 non-null   object 
 7   total_records              9762 non-null   int64  
 8   date                       9762 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 686.5+ KB
None


array(['ROAD_CLOSED', 'WEATHERHAZARD', 'JAM', 'ACCIDENT'], dtype=object)

In [32]:
# Simpan 3 sample jalan : N9 Raden KH Abdullah bin Nuh, N9 Jalan Raya Pajajaran, N9 Jalan Raya Dramaga
for i, street in enumerate(['N9 Raden KH Abdullah bin Nuh', 'N9 Jalan Raya Pajajaran', 'N9 Jalan Raya Dramaga']):
  print(i, street)
  df_alerts_sample = df_alerts.loc[(df_alerts['street'] == street)]
  df_alerts_sample = df_alerts_sample.drop(['id', 'kemendagri_kabupaten_kode', 'kemendagri_kabupaten_nama', 'street'
  , 'avg_location', 'total_records', 'date'], axis=1)
  df_alerts_sample['time'] = pd.to_datetime(df_alerts_sample['time'])
  # df_jams_agg = df_alerts_sample.groupby(pd.Grouper(key="time", freq='1H')).mean()
  df_alerts_sample.to_csv(r'D:\DSLS\Mini project\DS\clean_code\data\transformed\alerts\sample_alerts_{}.csv'.format(i), index=False)

0 N9 Raden KH Abdullah bin Nuh
1 N9 Jalan Raya Pajajaran
2 N9 Jalan Raya Dramaga


## irregularities

In [3]:
df_irregularities = pd.read_csv(median_irregularities_path)
df_irregularities.head(3)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,jam_level,median_length,median_delay_seconds,median_regular_speed,total_records,cause_type,median_seconds,median_speed,date,median_jam_level,id,geometry
0,2022-07-06 09:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Dramaga,4,1878.0,689.0,13.775,18,NaN,856.5,7.340,2022-07-06,4.0,1492793,"MULTILINESTRING ((106.738284 -6.568848, 106.73..."
1,2022-07-06 09:00:00.000,32.71,KOTA BOGOR,N9 Jalan Raya Dramaga,3,1226.0,339.5,14.400,6,NaN,447.0,9.995,2022-07-06,3.0,1492792,"MULTILINESTRING ((106.73546 -6.565547, 106.735..."
2,2022-07-06 10:00:00.000,32.71,KOTA BOGOR,Kapten Yusuf,3,1399.0,517.0,18.150,2,NaN,707.0,7.110,2022-07-06,3.0,1492794,"LINESTRING (106.785538 -6.61574, 106.785797 -6..."


In [4]:
# analisis null value
df_irregularities.isna().sum().plot(kind='bar')

In [5]:
# analisis statistik
df_irregularities.describe()

,kemendagri_kabupaten_kode,jam_level,median_length,median_delay_seconds,median_regular_speed,total_records,cause_type,median_seconds,median_speed,median_jam_level,id
count,4.051000e+03,4051.000000,4051.000000,4051.000000,4051.000000,4051.000000,0.0,4051.000000,4051.000000,4051.000000,4.051000e+03
mean,3.271000e+01,3.542582,1232.675389,567.778697,20.019296,15.730190,NaN,688.843125,7.244319,3.542582,1.543578e+06
std,7.106305e-15,0.603571,732.053520,280.972259,11.149602,17.365754,NaN,297.049553,4.563263,0.603571,2.515182e+04
min,3.271000e+01,1.000000,500.000000,60.000000,4.050000,1.000000,NaN,118.000000,0.880000,1.000000,1.492792e+06
25%,3.271000e+01,3.000000,747.000000,381.500000,14.017500,4.000000,NaN,493.000000,4.280000,3.000000,1.524546e+06
50%,3.271000e+01,4.000000,1072.000000,541.000000,18.345001,10.000000,NaN,661.000000,6.220000,4.000000,1.546230e+06
75%,3.271000e+01,4.000000,1420.000000,701.000000,23.277501,21.000000,NaN,839.000000,8.920000,4.000000,1.566736e+06
max,3.271000e+01,4.000000,7105.000000,2588.500000,92.710000,126.000000,NaN,2676.500000,51.425000,4.000000,1.583286e+06


In [7]:
# analisis kolom
print(df_irregularities.info())
df_irregularities['median_jam_level'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4051 entries, 0 to 4050
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   time                       4051 non-null   object 
 1   kemendagri_kabupaten_kode  4051 non-null   float64
 2   kemendagri_kabupaten_nama  4051 non-null   object 
 3   street                     4051 non-null   object 
 4   jam_level                  4051 non-null   int64  
 5   median_length              4051 non-null   float64
 6   median_delay_seconds       4051 non-null   float64
 7   median_regular_speed       4051 non-null   float64
 8   total_records              4051 non-null   int64  
 9   cause_type                 0 non-null      float64
 10  median_seconds             4051 non-null   float64
 11  median_speed               4051 non-null   float64
 12  date                       4051 non-null   object 
 13  median_jam_level           4051 non-null   float

array([4., 3., 2., 1.])

In [8]:
# no null, all row are ok
for i, street in enumerate(['N9 Raden KH Abdullah bin Nuh', 'N9 Jalan Raya Pajajaran', 'N9 Jalan Raya Dramaga']):
  print(i, street)
  df_irr_sample = df_irregularities.loc[(df_irregularities['street'] == street)]
  df_irr_sample = df_irr_sample.drop(['id', 'kemendagri_kabupaten_kode', 'kemendagri_kabupaten_nama', 'street'
  , 'jam_level', 'total_records', 'cause_type', 'date', 'id', 'geometry'], axis=1)
  df_irr_sample['time'] = pd.to_datetime(df_irr_sample['time'])
  # df_irr_agg = df_irr_sample.groupby(pd.Grouper(key="time", freq='1H')).mean()
  df_irr_sample.to_csv(r'D:\DSLS\Mini project\DS\clean_code\data\transformed\irregularities\sample_irr_{}.csv'.format(i))

0 N9 Raden KH Abdullah bin Nuh
1 N9 Jalan Raya Pajajaran
2 N9 Jalan Raya Dramaga


# Null handling setiap jalan

## jalan 0

In [13]:
df_sample_jams = pd.read_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_0.csv')
df_sample_jams.isnull().sum()

time                  0
median_length       337
median_delay        337
median_speed_kmh    337
median_level        337
dtype: int64

In [14]:
df_sample_jams = pd.read_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_0.csv')
# df_sample_jams.isnull().sum()
df_sample_jams['time'] = pd.to_datetime(df_sample_jams['time'])
df_sample_jams = df_sample_jams.set_index('time')
total = 0
for i in range(0, 24):
  total_row = len(df_sample_jams[df_sample_jams.index.hour == i])
  null_value = df_sample_jams[df_sample_jams.index.hour == i].isna().sum()[1]
  total += null_value
  print(f"jam {i}, null value: {null_value}/{total_row}")
print('total: ', total)

jam 0, null value: 45/60
jam 1, null value: 52/60
jam 2, null value: 53/60
jam 3, null value: 53/60
jam 4, null value: 50/60
jam 5, null value: 36/60
jam 6, null value: 7/61
jam 7, null value: 2/61
jam 8, null value: 1/61
jam 9, null value: 0/61
jam 10, null value: 0/61
jam 11, null value: 0/61
jam 12, null value: 0/61
jam 13, null value: 0/61
jam 14, null value: 0/61
jam 15, null value: 0/61
jam 16, null value: 0/61
jam 17, null value: 0/61
jam 18, null value: 0/61
jam 19, null value: 0/61
jam 20, null value: 1/61
jam 21, null value: 1/61
jam 22, null value: 8/61
jam 23, null value: 28/61
total:  337


In [15]:
def clean_df(df_jams_agg):
  list_index = []
  for i in [0, 1, 2, 3, 4, 5, 23]:
    list_index.extend(df_jams_agg[df_jams_agg.index.hour == i].index)
  print(str(len(list_index))+' data cleaned')
  df_jams_agg_clean = df_jams_agg.drop(list_index)
  for i in [6,7,8,20,21,22]:
    df_null = df_jams_agg_clean[df_jams_agg_clean.index.hour == i]
    replaced_index = df_null[df_null.median_length.isnull()].index
    median_values = []
    for col in df_null.columns:
      if col == 'median_level':
        median_values.append(round(df_null[col].mean(), 0))
        continue
      median_values.append(df_null[col].median())
    for idx in replaced_index:
      for col_idx, col_name in enumerate(df_null.columns):
        df_jams_agg_clean.loc[idx, col_name] = median_values[col_idx]
  df_jams_agg_clean['median_level'] = df_jams_agg_clean['median_level'].astype(int)
  return df_jams_agg_clean

df_jams_agg_clean = clean_df(df_sample_jams)
df_jams_agg_clean
df_jams_agg_clean.to_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_cleaned_0.csv')

421 data cleaned


## jalan 1

In [11]:
df_sample_jams = pd.read_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_1.csv')
df_sample_jams.isnull().sum()

time                  0
median_length       301
median_delay        301
median_speed_kmh    301
median_level        301
dtype: int64

In [32]:
df_sample_jams.head(28)

,time,median_length,median_delay,median_speed_kmh,median_level
0,2022-07-06 01:00:00,808.00,73.00,19.920000,2.0
1,2022-07-06 02:00:00,NaN,NaN,NaN,NaN
2,2022-07-06 03:00:00,NaN,NaN,NaN,NaN
3,2022-07-06 04:00:00,NaN,NaN,NaN,NaN
4,2022-07-06 05:00:00,NaN,NaN,NaN,NaN
5,2022-07-06 06:00:00,NaN,NaN,NaN,NaN
6,2022-07-06 07:00:00,1096.00,105.50,17.529999,2.5
7,2022-07-06 08:00:00,1152.75,94.25,20.830000,2.5
8,2022-07-06 09:00:00,1144.00,96.00,16.570000,2.0
9,2022-07-06 10:00:00,1174.00,83.50,22.230000,2.0


In [12]:
df_sample_jams = pd.read_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_1.csv')
# df_sample_jams.isnull().sum()
df_sample_jams['time'] = pd.to_datetime(df_sample_jams['time'])
df_sample_jams = df_sample_jams.set_index('time')
total = 0
for i in range(0, 24):
  total_row = len(df_sample_jams[df_sample_jams.index.hour == i])
  null_value = df_sample_jams[df_sample_jams.index.hour == i].isna().sum()[1]
  total += null_value
  print(f"jam {i}, null value: {null_value}/{total_row}")
print('total: ', total)

jam 0, null value: 31/62
jam 1, null value: 38/62
jam 2, null value: 42/62
jam 3, null value: 48/62
jam 4, null value: 50/62
jam 5, null value: 39/62
jam 6, null value: 8/62
jam 7, null value: 2/62
jam 8, null value: 1/62
jam 9, null value: 1/62
jam 10, null value: 1/62
jam 11, null value: 1/62
jam 12, null value: 1/62
jam 13, null value: 1/62
jam 14, null value: 1/62
jam 15, null value: 1/62
jam 16, null value: 1/62
jam 17, null value: 1/62
jam 18, null value: 1/62
jam 19, null value: 1/62
jam 20, null value: 1/62
jam 21, null value: 2/62
jam 22, null value: 6/62
jam 23, null value: 22/62
total:  301


In [8]:
df_sample_jams = df_sample_jams.drop(df_sample_jams[df_sample_jams.index.date == datetime.date(2022,9,5)].index)
df_sample_jams = df_sample_jams.drop(df_sample_jams[df_sample_jams.index.date == datetime.date(2022,9,6)].index)
df_sample_jams.tail()

,median_length,median_delay,median_speed_kmh,median_level
time,,,,
2022-09-04 19:00:00,1412.5,119.00,19.592500,2.5
2022-09-04 20:00:00,1545.0,111.00,21.115000,2.5
2022-09-04 21:00:00,1695.5,81.50,28.312501,1.5
2022-09-04 22:00:00,284.0,66.50,8.405001,3.0
2022-09-04 23:00:00,966.5,101.75,16.167500,2.5


In [10]:
def clean_df(df_jams_agg):
  list_index = []
  for i in [0, 1, 2, 3, 4, 5, 23]:
    list_index.extend(df_jams_agg[df_jams_agg.index.hour == i].index)
  print(str(len(list_index))+' data cleaned')
  df_jams_agg_clean = df_jams_agg.drop(list_index)
  for i in [6,7,8,20,21,22]:
    df_null = df_jams_agg_clean[df_jams_agg_clean.index.hour == i]
    replaced_index = df_null[df_null.median_length.isnull()].index
    median_values = []
    for col in df_null.columns:
      if col == 'median_level':
        median_values.append(round(df_null[col].mean(), 0))
        continue
      median_values.append(df_null[col].median())
    for idx in replaced_index:
      for col_idx, col_name in enumerate(df_null.columns):
        df_jams_agg_clean.loc[idx, col_name] = median_values[col_idx]
  df_jams_agg_clean['median_level'] = df_jams_agg_clean['median_level'].astype(int)
  return df_jams_agg_clean

df_jams_agg_clean = clean_df(df_sample_jams)
df_jams_agg_clean
df_jams_agg_clean.to_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_cleaned_1.csv')

426 data cleaned


## jalan 2

In [16]:
df_sample_jams = pd.read_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_2.csv')
df_sample_jams.isnull().sum()

time                  0
median_length       391
median_delay        391
median_speed_kmh    391
median_level        391
dtype: int64

In [17]:
df_sample_jams = pd.read_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_2.csv')
df_sample_jams['time'] = pd.to_datetime(df_sample_jams['time'])
df_sample_jams = df_sample_jams.set_index('time')
total = 0
for i in range(0, 24):
  total_row = len(df_sample_jams[df_sample_jams.index.hour == i])
  null_value = df_sample_jams[df_sample_jams.index.hour == i].isna().sum()[1]
  total += null_value
  print(f"jam {i}, null value: {null_value}/{total_row}")
print('total: ', total)

jam 0, null value: 48/60
jam 1, null value: 58/60
jam 2, null value: 59/60
jam 3, null value: 58/60
jam 4, null value: 57/60
jam 5, null value: 47/60
jam 6, null value: 13/61
jam 7, null value: 2/61
jam 8, null value: 2/61
jam 9, null value: 0/61
jam 10, null value: 0/61
jam 11, null value: 0/61
jam 12, null value: 0/61
jam 13, null value: 0/61
jam 14, null value: 0/61
jam 15, null value: 0/61
jam 16, null value: 0/61
jam 17, null value: 0/61
jam 18, null value: 0/61
jam 19, null value: 0/61
jam 20, null value: 1/61
jam 21, null value: 1/61
jam 22, null value: 12/61
jam 23, null value: 33/61
total:  391


In [18]:
def clean_df(df_jams_agg):
  list_index = []
  for i in [0, 1, 2, 3, 4, 5, 23]:
    list_index.extend(df_jams_agg[df_jams_agg.index.hour == i].index)
  print(str(len(list_index))+' data cleaned')
  df_jams_agg_clean = df_jams_agg.drop(list_index)
  for i in [6,7,8,20,21,22]:
    df_null = df_jams_agg_clean[df_jams_agg_clean.index.hour == i]
    replaced_index = df_null[df_null.median_length.isnull()].index
    median_values = []
    for col in df_null.columns:
      if col == 'median_level':
        median_values.append(round(df_null[col].mean(), 0))
        continue
      median_values.append(df_null[col].median())
    for idx in replaced_index:
      for col_idx, col_name in enumerate(df_null.columns):
        df_jams_agg_clean.loc[idx, col_name] = median_values[col_idx]
  df_jams_agg_clean['median_level'] = df_jams_agg_clean['median_level'].astype(int)
  return df_jams_agg_clean

df_jams_agg_clean = clean_df(df_sample_jams)
df_jams_agg_clean
df_jams_agg_clean.to_csv(r'D:\DSLS\Mini project\DS\results\jams\sample_jams_cleaned_2.csv')

421 data cleaned
